In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import seaborn as sns
#%pylab inline

In [27]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-12-19/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 3027
files_end = 3142

In [28]:
# cup configuration
cup_config = cycle_Sb_2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
#corr_isotopes_Sb = {"Te": "125", "Xe" : "129"}
corr_isotopes_Sb = {}
#
# Mass Range of cup configuration
mass_range = cycle_Sb_2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"], ["117", "118", "119", "122", "124"]]
#isotopes = [["112", "114", "115", "116", "117", "118", "119", "122", "124"]]
isotopes = [["116", "117", "118", "119", "122", "124"]]
isotopes_column = ["116", "117", "118", "119", "122", "124"]
data_sample_column = [(i + "/" + "120") for i in isotopes_column]

denom_isotope = "120"
norm_ratio = ["122","120"]
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False

blk_corr = False

In [29]:
columns = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)"]
blk_ls = []
sample_ls = []
df_all = pd.DataFrame()
files_1 = range(files_start, files_end, 1)
for sample in files_1:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero["sample"] = sample_name
    df_zero["cycle"] = df_zero.index.values
    df_zero.index = df_zero["date"]
    df_zero = df_zero[columns]
    df_all = df_all.append(df_zero, ignore_index = True)

    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "teflon blank" or sample_name == "blank 3_1" or sample_name == "col blk 271115":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1)
date,,,,,,,,,,,,,,
2015-12-20 11:12:00,1,NIST 100ppb,0.013977,0.016953,-7.113900e-06,2.297109,5.199604,1.793174,6.779303,12.309710,3.201959,8.915184,2.787076,5.214065
2015-12-20 11:12:00,2,NIST 100ppb,0.013979,0.016942,-4.268200e-06,2.315225,5.240730,1.806998,6.832764,12.405612,3.226805,8.984167,2.808422,5.254299
2015-12-20 11:12:00,3,NIST 100ppb,0.013991,0.016998,2.845560e-05,2.311728,5.232602,1.804236,6.821781,12.384179,3.221402,8.969213,2.804007,5.244739
2015-12-20 11:12:00,4,NIST 100ppb,0.014146,0.017211,-1.280510e-05,2.302407,5.210806,1.796897,6.792392,12.331702,3.207018,8.928181,2.790668,5.220520
2015-12-20 11:12:00,5,NIST 100ppb,0.014017,0.017064,-7.113000e-07,2.300400,5.207931,1.795511,6.790037,12.323513,3.205351,8.924358,2.789649,5.219053
2015-12-20 11:12:00,6,NIST 100ppb,0.013894,0.016849,-7.113900e-06,2.295590,5.197560,1.792127,6.777354,12.302043,3.200370,8.910389,2.785803,5.211368
2015-12-20 11:12:00,7,NIST 100ppb,0.013892,0.016816,3.556960e-05,2.279604,5.162284,1.779816,6.731403,12.217534,3.178180,8.848948,2.766362,5.175548
2015-12-20 11:12:00,8,NIST 100ppb,0.013892,0.016843,-1.493930e-05,2.211972,5.007608,1.726842,6.529160,11.852820,3.083450,8.585703,2.684145,5.020914
2015-12-20 11:12:00,9,NIST 100ppb,0.013899,0.016844,-1.778480e-05,2.202166,4.986989,1.719104,6.502578,11.802014,3.069843,8.547053,2.671808,4.999546


In [30]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_external_norm = pd.DataFrame()
df_external_norm_mean = pd.DataFrame()
df_external_outlier = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)

# arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample = True
    
#blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False    
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    data_sample = new_corr.internal_norm_1(norm_ratio, denom_isotope, iter_beta)
# new method for df with all data together with signals in pycharm
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = new_corr.mad_outlier_rejection(data_sample)
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample.drop(denom_isotope, axis=1)
    data_sample = data_sample[sorted(data_sample.columns)]
    data_sample.columns = data_sample_column
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    #data_sample_outlier = data_sample_outlier[["116", "117", "119", "120", "122", "124"]]
    #data_sample_outlier.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    #data_sample_outlier["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    #data_sample_outlier["sample"] = sample_name
    #data_sample_outlier["cycle"] = data_sample.index.values
    
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_external_norm = df_external_norm.append(data_sample)
    #df_external_outlier = df_external_outlier.append(data_sample_outlier) 
    print sample, sample_name

3027 NIST 100ppb
3029 NIST 100ppb
3030 NIST 100ppb
3031 NIST 100ppb
3032 NIST 100ppb
3033 NIST 100ppb
3035 NIST 100ppb
3036 NIST 100ppb
3037 NIST 100ppb
3038 NIST 100ppb
3039 NIST 100ppb
3042 NIST 100ppb
3045 NIST 100ppb
3047 NIST 100ppb
3048 NIST 100ppb
3049 NIST 100ppb
3050 NIST 100ppb
3051 NIST 100ppb
3053 NIST 100ppb
3054 NIST 100ppb
3055 NIST 100ppb
3056 NIST 100ppb
3057 NIST 100ppb
3059 NIST 100ppb
3060 SPEX 100ppb
3061 NIST 100ppb
3062 SPEX 100ppb
3063 NIST 100ppb
3065 NIST 100ppb
3066 NIST 3_1
3067 NIST 100ppb
3068 NIST 4_1
3069 NIST 100ppb
3071 NIST 100ppb
3072 J2 2_1(2)
3073 NIST 100ppb
3074 J2 3_1(1)
3075 NIST 100ppb
3077 NIST 100ppb
3078 NIST 100ppb
3079 NIST 100ppb
3080 NIST 100ppb
3081 NIST 100ppb
3083 NIST 100ppb
3084 ZH09-05 23.3_1_2(1)
3085 NIST 100ppb
3086 ZH09-05 33.8_1(1)
3087 NIST 100ppb
3089 NIST 100ppb
3090 ZH09-05 44.8_1(1)
3091 NIST 100ppb
3092 Jenny 12-06W 57_1(1)
3093 NIST 100ppb
3095 NIST 100ppb
3096 NIST 100ppb
3097 NIST 100ppb
3098 NIST 100ppb
3099 NIST 10

In [69]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_external_norm = pd.DataFrame()
df_external_norm_mean = pd.DataFrame()
df_external_outlier = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # method for blank correction
    # arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample == True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False  
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
    
    data_sample_raw = new_corr.raw_signals("118")
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = data_sample_raw[np.abs(data_sample_raw-data_sample_raw.mean())<=(2*data_sample_raw.std())]
    
    data_sample = new_corr.external_norm_Sb("118", 10)
# new method for df with all data together with signals in pycharm
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = data_sample[np.abs(data_sample-data_sample.mean())<=(2*data_sample.std())]
    data_sample_outlier= data_sample[np.abs(data_sample-data_sample.mean())>(2*data_sample.std())]
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample[["116", "117", "119", "120", "122", "124"]]
    data_sample.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    data_sample_outlier = data_sample_outlier[["116", "117", "119", "120", "122", "124"]]
    data_sample_outlier.columns = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
    data_sample_outlier["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample_outlier["sample"] = sample_name
    data_sample_outlier["cycle"] = data_sample.index.values
    
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_external_norm = df_external_norm.append(data_sample)
    df_external_outlier = df_external_outlier.append(data_sample_outlier) 

In [34]:
#df_external_norm.set_index("date", inplace = True)
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_external_sub = pd.DataFrame(df_external_norm, columns = columns)

#df_all_sub = df_all[(df_all["sample"] != "blank sol") & (df_all["sample"] != "teflon blank") & (sample_name != "blank 3_1") & (sample_name != "col blk 271115") & (sample_name != "SQ")]
#df_external_sub["125"] = df_all_sub["H6 (1)"]
#df_external_sub["Te/Sn"] = (df_all_sub["H6 (1)"]/7.0509)/(df_all_sub["L1 (1)"]/24.223)

#df_external_sub = df_external_sub[df_external_sub["118"] > 4]
df_external_sub

,cycle,sample,116/120,117/120,118/120,119/120,122/120,124/120,116,117,118,119,120,122,124
date,,,,,,,,,,,,,,,
2015-12-20 11:12:00,1,NIST 100ppb,0.445776,0.235215,0.742821,0.263414,0.142086,0.177607,5.214065,2.787076,8.915184,3.201959,12.309710,1.793174,2.297109
2015-12-20 11:12:00,2,NIST 100ppb,0.445669,0.235155,0.742721,0.263395,0.142086,0.177652,5.254299,2.808422,8.984167,3.226805,12.405612,1.806998,2.315225
2015-12-20 11:12:00,3,NIST 100ppb,0.445810,0.235263,0.742919,0.263435,0.142086,0.177621,5.244739,2.804007,8.969213,3.221402,12.384179,1.804236,2.311728
2015-12-20 11:12:00,4,NIST 100ppb,0.445796,0.235202,0.742796,0.263398,0.142086,0.177597,5.220520,2.790668,8.928181,3.207018,12.331702,1.796897,2.302407
2015-12-20 11:12:00,5,NIST 100ppb,0.445869,0.235234,0.742890,0.263421,0.142086,0.177598,5.219053,2.789649,8.924358,3.205351,12.323513,1.795511,2.300400
2015-12-20 11:12:00,6,NIST 100ppb,0.445859,0.235268,0.742914,0.263452,0.142086,0.177586,5.211368,2.785803,8.910389,3.200370,12.302043,1.792127,2.295590
2015-12-20 11:12:00,7,NIST 100ppb,0.445857,0.235242,0.742894,0.263435,0.142086,0.177570,5.175548,2.766362,8.848948,3.178180,12.217534,1.779816,2.279604
2015-12-20 11:12:00,8,NIST 100ppb,0.445928,0.235307,0.743042,0.263459,0.142086,0.177571,5.020914,2.684145,8.585703,3.083450,11.852820,1.726842,2.211972
2015-12-20 11:12:00,9,NIST 100ppb,0.445763,0.235163,0.742734,0.263400,0.142086,0.177614,4.999546,2.671808,8.547053,3.069843,11.802014,1.719104,2.202166


In [35]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

df_external_norm_mean = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).mean()
df_external_norm_mean_sd = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).agg([np.mean, SE])
df_external_norm_mean["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]
df_external_norm_mean_sd["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]

#df_external_norm_mean = df_external_norm_mean[1:]
#df_external_norm_mean = df_external_norm_mean.drop(datetime.strptime("20150310 23:59", "%Y%m%d %H:%M"))
#df_external_norm_mean_sd = df_external_norm_mean_sd[1:]
df_external_norm_mean

,116/120,117/120,118/120,119/120,122/120,124/120,116,117,118,119,120,122,124,sample
date,,,,,,,,,,,,,,
2015-12-20 11:12:00,0.445827,0.235233,0.742853,0.263430,0.142086,0.177600,5.120155,2.736908,8.754787,3.144553,12.088810,1.763940,2.256190,NIST 100ppb
2015-12-20 11:27:00,0.445889,0.235267,0.742891,0.263448,0.142086,0.177581,5.119973,2.736974,8.754642,3.144730,12.089275,1.761405,2.256492,NIST 100ppb
2015-12-20 11:36:00,0.445905,0.235272,0.742897,0.263455,0.142086,0.177575,5.246180,2.804424,8.970253,3.222238,12.386986,1.804786,2.312017,NIST 100ppb
2015-12-20 11:45:00,0.445907,0.235283,0.742892,0.263455,0.142086,0.177572,5.121299,2.737560,8.755594,3.144980,12.089250,1.761209,2.255909,NIST 100ppb
2015-12-20 11:53:00,0.445931,0.235288,0.742914,0.263463,0.142086,0.177566,5.060001,2.704794,8.650902,3.107427,11.944783,1.740219,2.229026,NIST 100ppb
2015-12-20 12:02:00,0.445916,0.235282,0.742899,0.263458,0.142086,0.177570,5.274236,2.819339,9.017294,3.239030,12.450925,1.813967,2.323535,NIST 100ppb
2015-12-20 12:17:00,0.445910,0.235282,0.742890,0.263458,0.142086,0.177570,5.244051,2.803267,8.965877,3.220632,12.367260,1.803697,2.307992,NIST 100ppb
2015-12-20 12:26:00,0.445907,0.235278,0.742885,0.263459,0.142086,0.177574,5.243783,2.803083,8.965372,3.220489,12.379631,1.803604,2.310350,NIST 100ppb
2015-12-20 12:35:00,0.445933,0.235291,0.742909,0.263464,0.142086,0.177565,5.116087,2.734837,8.746814,3.141912,12.077273,1.759527,2.253739,NIST 100ppb


In [11]:
plot_Nist = df_external_norm_mean[df_external_norm_mean["sample"] == "NIST 100ppb"]
plot_Nist = df_external_norm_mean[(df_external_norm_mean["118"] >=1)]
#dataplot = plot(plot_Nist["Te/Sn"], plot_Nist["124/120"], "ro")
dataplot = plot(plot_Nist.index, plot_Nist["122/120"], "ro")
show(dataplot)

In [36]:
df_external_norm_mean = df_external_norm_mean[(df_external_norm_mean["118"] >=1)]
sub_index = df_external_norm_mean[(df_external_norm_mean["sample"].shift(1) == df_external_norm_mean["sample"].shift(-1)) & (df_external_norm_mean["sample"].shift(1) == "NIST 100ppb")].index
sample = df_external_norm_mean["sample"]
new = df_external_norm_mean[df_external_norm_mean.columns[df_external_norm_mean.dtypes != 'object']]

new =  ((new / ((new.shift(1) + new.shift(-1))/2))-1)*10000
new["sample"] = sample

new = new[new.index.isin(sub_index)]
#df_external_norm_signal = df_external_norm_signal[df_external_norm_signal['sample'].isin(sub_index.tolist())]
#df_new_signal = new.merge(df_external_norm_signal, left_index=True, right_index=True)
#new[new.sample != "NIST 100ppb"]
new

,116/120,117/120,118/120,119/120,122/120,124/120,116,117,118,119,120,122,124,sample
date,,,,,,,,,,,,,,
2015-12-20 11:27:00,0.510819,0.617528,0.207144,0.202255,0,-0.376816,-121.922254,-121.602910,-121.723345,-121.460420,-121.445235,-128.663972,-120.886616,NIST 100ppb
2015-12-20 11:36:00,0.165828,-0.149057,0.084055,0.128361,0,-0.106977,245.173226,245.343001,245.724464,246.062976,246.271298,246.855143,247.390345,NIST 100ppb
2015-12-20 11:45:00,-0.251718,0.150390,-0.187216,-0.128040,0,0.119019,-61.694545,-61.892051,-62.406281,-62.728524,-62.991176,-63.714299,-64.354965,NIST 100ppb
2015-12-20 11:53:00,0.440232,0.228823,0.248346,0.234389,0,-0.291870,-265.049540,-265.095498,-265.057563,-264.969368,-265.120415,-264.989393,-265.081632,NIST 100ppb
2015-12-20 12:02:00,-0.099769,-0.137819,-0.040081,-0.099030,0,0.110962,237.207713,237.137292,237.165185,237.043132,242.599085,237.080219,242.564694,NIST 100ppb
2015-12-20 12:17:00,-0.045378,0.104162,-0.022023,-0.021224,0,-0.108528,-28.442931,-28.257265,-28.311248,-28.262254,-38.676552,-28.135665,-38.630269,NIST 100ppb
2015-12-20 12:26:00,-0.309705,-0.389556,-0.199550,-0.079882,0,0.355936,122.997524,122.896842,123.105208,123.276100,128.752471,123.440590,129.268345,NIST 100ppb
2015-12-20 12:35:00,0.508756,0.464390,0.270532,0.149154,0,-0.423253,-287.462446,-287.366467,-287.531128,-287.648472,-287.754767,-287.715220,-288.108843,NIST 100ppb
2015-12-20 12:43:00,-0.199842,-0.140055,-0.069444,-0.084084,0,0.296724,259.838543,260.090640,260.352971,260.529247,260.793294,261.156165,261.823429,NIST 100ppb


In [37]:
def SD2(x):
    SD2 = np.std(x) * 2
    return SD2

#new = new.drop(["116", "117", "118", "119", "120", "122", "124"], axis=1)
np.round(new.groupby("sample").agg([np.mean, SD2]), decimals=2)

116/120       117/120       118/120       119/120        \
                        mean   SD2    mean   SD2    mean   SD2    mean   SD2   
sample                                                                         
J2 2_1(2)               0.20  0.00   -0.13  0.00   -0.02  0.00    0.13  0.00   
J2 3_1(1)              -6.07  0.00   -6.13  0.00   -4.73  0.00   -2.26  0.00   
Jenny 12-06W 57_1(1)   -0.16  0.00   -0.25  0.00    0.06  0.00   -0.05  0.00   
NIST 100ppb             0.02  0.56    0.01  0.51    0.00  0.28    0.01  0.29   
NIST 3_1               -0.15  0.00   -0.17  0.00   -0.03  0.00    0.10  0.00   
NIST 4_1                0.08  0.00    0.10  0.00    0.02  0.00   -0.01  0.00   
SPEX 100ppb            -0.17  0.73   -0.10  0.60   -0.07  0.41   -0.08  0.30   
ZH09-05 10.3_1(1)      -0.06  0.00   -0.22  0.00    0.05  0.00   -0.16  0.00   
ZH09-05 23.3_1_1(1)     3.27  0.00   -0.17  0.00    0.04  0.00   -0.12  0.00   
ZH09-05 23.3_1_2(1)     0.47  0.00   -0.00  0.00    0.06  0.00   -0.06  0.00   
ZH09-05 33.8_1(1)       0.51  0.00   -0.10  0.00    0.07  0.00    0.06  0.00   
ZH09-05 44.8_1(1)       0.03  0.00   -0.00  0.00    0.07  0.00   -0.03  0.00   
ZH09-05 50.5_1(1)      -0.28  0.00   -0.38  0.00   -0.02  0.00   -0.03  0.00   
ZH09-05 56.7_1(1)      -0.33  0.00   -0.36  0.00   -0.10  0.00   -0.04  0.00   

                     122/120       ...        118              119          \
                        mean SD2   ...       mean     SD2     mean     SD2   
sample                             ...                                       
J2 2_1(2)                  0   0   ...     642.73    0.00   643.74    0.00   
J2 3_1(1)                  0   0   ...   -7958.50    0.00 -7958.07    0.00   
Jenny 12-06W 57_1(1)       0   0   ...     898.92    0.00   899.41    0.00   
NIST 100ppb                0   0   ...      -6.10  352.01    -6.21  351.58   
NIST 3_1                   0   0   ...     924.77    0.00   924.69    0.00   
NIST 4_1                   0   0   ...     859.77    0.00   858.88    0.00   
SPEX 100ppb                0   0   ...      52.85  359.93    53.27  361.19   
ZH09-05 10.3_1(1)          0   0   ...     939.29    0.00   941.93    0.00   
ZH09-05 23.3_1_1(1)        0   0   ...     978.70    0.00   986.36    0.00   
ZH09-05 23.3_1_2(1)        0   0   ...     825.47    0.00   825.31    0.00   
ZH09-05 33.8_1(1)          0   0   ...    1095.53    0.00  1095.95    0.00   
ZH09-05 44.8_1(1)          0   0   ...     749.95    0.00   748.65    0.00   
ZH09-05 50.5_1(1)          0   0   ...    1959.29    0.00  1958.85    0.00   
ZH09-05 56.7_1(1)          0   0   ...     947.34    0.00   947.97    0.00   

                          120              122              124          
                         mean     SD2     mean     SD2     mean     SD2  
sample                                                                   
J2 2_1(2)              644.46    0.00   646.13    0.00   647.78    0.00  
J2 3_1(1)            -7957.64    0.00 -7957.73    0.00 -7957.00    0.00  
Jenny 12-06W 57_1(1)   900.09    0.00   901.42    0.00   902.63    0.00  
NIST 100ppb             -6.02  352.60    -6.15  352.08    -6.04  352.79  
NIST 3_1               924.36    0.00   923.92    0.00   923.51    0.00  
NIST 4_1               858.04    0.00   856.36    0.00   854.59    0.00  
SPEX 100ppb             54.39  360.33    56.63  360.30    57.69  361.75  
ZH09-05 10.3_1(1)      944.91    0.00   950.51    0.00   956.17    0.00  
ZH09-05 23.3_1_1(1)    994.25    0.00  1009.61    0.00  1024.79    0.00  
ZH09-05 23.3_1_2(1)    825.31    0.00   825.18    0.00   824.86    0.00  
ZH09-05 33.8_1(1)     1096.28    0.00  1097.13    0.00  1097.58    0.00  
ZH09-05 44.8_1(1)      747.59    0.00   745.41    0.00   742.97    0.00  
ZH09-05 50.5_1(1)     1958.45    0.00  1957.66    0.00  1951.36    0.00  
ZH09-05 56.7_1(1)      948.59    0.00   949.70    0.00   951.00    0.00  

[14 rows x 26 columns]

In [202]:
df_all.to_csv(path_sav + "Sn_H8_L3_Sb_NU_Signals_zero_corr.csv", mode='a', header=False)

In [100]:
df_external_sub.to_csv(path_sav + "Sn_H6_L3_SB_external_norm.csv", mode='a', header=False)

In [13]:
df_external_norm_mean_sd.to_csv(path + "Sn_H8_L3_SB_internal_norm_116_120_MAD.csv", mode='w', header=True)